In [3]:
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup 

In [30]:
HEADERS = {"User-Agent": "Your Name your.email@example.com"}

# Apple's CIK is 0000320193 (must be 10 digits, zero-padded)
cik = "0000320193"

# 1. Get submissions for this company
url = f"https://data.sec.gov/submissions/CIK{cik}.json"
resp = requests.get(url, headers=HEADERS)
data = resp.json()

# 2. Filter only Form 4 filings
form4_filings = []
for filing, form in zip(data["filings"]["recent"]["accessionNumber"], 
                        data["filings"]["recent"]["form"]):
    if form == "4":
        form4_filings.append(filing)

print("Recent Form 4 filings for Apple:")
for f in form4_filings[:5]:
    print(f)

# 3. Grab one of the raw filings in XML
accession = form4_filings[0].replace("-", "")
xml_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession}/"
xml_resp = requests.get(xml_url, headers=HEADERS)

soup1 = BeautifulSoup(xml_resp.text, 'xml')
xml_link_tag = soup1.find('a', href=lambda href: href and href.endswith('.xml'))
if xml_link_tag:
    xml_file_url = xml_link_tag['href']
    soup = BeautifulSoup(requests.get(f"https://www.sec.gov/{xml_file_url}", headers=HEADERS).text, 'xml')  
else:
    print("No link ending with .xml was found.")

# --- Stakeholder info ---
owner_name = soup.find("rptOwnerName").text
relationship = soup.find("reportingOwnerRelationship")
roles = []
if relationship.find("isDirector").text == "1":
    roles.append("Director")
if relationship.find("isOfficer").text == "1":
    roles.append(relationship.find("officerTitle").text)
if relationship.find("isTenPercentOwner").text == "1":
    roles.append("10% Owner")
print(f"Stakeholder: {owner_name}")
print("Relationship:", ", ".join(roles))
print("------")




Recent Form 4 filings for Apple:
0001214156-25-000011
0001767094-25-000009
0002078476-25-000005
0001462356-25-000010
0002050912-25-000006
Stakeholder: COOK TIMOTHY D
Relationship: Director, Chief Executive Officer



In [ ]:
# 4. Loop through the most recent 20 Form 4 filings and print reporting owner details
for idx, f in enumerate(form4_filings[:20]):
    print(f"Filing #{idx+1}: accession (raw): {f}")
    accession = f.replace("-", "")
    xml_index_url = f"https://data.sec.gov/submissions/CIK{int(cik)}.json"
    # The XML for a filing is usually available under the filing's directory. Construct expected path:
    xml_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession}/{accession}-index.html"

    # Try to fetch the filing directory listing first (index) and find XML link
    try:
        file_resp = requests.get(f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession}/", headers=HEADERS, timeout=10)
        file_resp.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"  Could not fetch filing directory: {e}")
        print("------")
        continue

    file_soup = BeautifulSoup(file_resp.text, 'html.parser')
    # look for .xml link
    xml_link_tag = file_soup.find('a', href=lambda href: href and href.endswith('.xml'))
    if not xml_link_tag:
        # sometimes the index page lists files differently; fallback to searching for files with .xml in text
        links = file_soup.find_all('a')
        xml_href = None
        for tag in links:
            href = tag.get('href', '')
            if '.xml' in href:
                xml_href = href
                break
        if xml_href:
            xml_file_url = xml_href
        else:
            print("  No XML file link found for this filing.")
            print("------")
            continue
    else:
        xml_file_url = xml_link_tag['href']

    # Ensure full URL
    if xml_file_url.startswith('/'):
        xml_full_url = f"https://www.sec.gov{xml_file_url}"
    elif xml_file_url.startswith('http'):
        xml_full_url = xml_file_url
    else:
        xml_full_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession}/{xml_file_url}"

    try:
        xml_resp = requests.get(xml_full_url, headers=HEADERS, timeout=10)
        xml_resp.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"  Could not fetch XML file: {e}")
        print("------")
        continue

    xml_soup = BeautifulSoup(xml_resp.text, 'xml')

    # There can be multiple reportingOwner entries
    rptOwners = xml_soup.find_all('reportingOwner')
    if not rptOwners:
        print("  No reportingOwner entries found in this XML.")
        print("------")
        continue

    for ro_idx, ro in enumerate(rptOwners):
        print(f"  Reporting Owner #{ro_idx+1}:")
        name_tag = ro.find('rptOwnerName')
        owner_name = name_tag.text.strip() if name_tag else 'N/A'
        owner_cik_tag = ro.find('rptOwnerCik')
        owner_cik = owner_cik_tag.text.strip() if owner_cik_tag else 'N/A'
        print(f"    Name: {owner_name}")
        print(f"    CIK: {owner_cik}")

        rel = ro.find('reportingOwnerRelationship')
        if rel:
            isDirector = rel.find('isDirector').text if rel.find('isDirector') else '0'
            isOfficer = rel.find('isOfficer').text if rel.find('isOfficer') else '0'
            officerTitle = rel.find('officerTitle').text if rel.find('officerTitle') else ''
            isTenPercentOwner = rel.find('isTenPercentOwner').text if rel.find('isTenPercentOwner') else '0'
            isOther = rel.find('isOther').text if rel.find('isOther') else '0'

            roles = []
            if isDirector == '1':
                roles.append('Director')
            if isOfficer == '1':
                roles.append(officerTitle or 'Officer')
            if isTenPercentOwner == '1':
                roles.append('10% Owner')
            if isOther == '1':
                roles.append('Other')

            print(f"    Relationship flags: isDirector={isDirector}, isOfficer={isOfficer}, isTenPercentOwner={isTenPercentOwner}, isOther={isOther}")
            print(f"    Roles: {', '.join(roles) if roles else 'None'}")
        else:
            print("    No reportingOwnerRelationship tag found")

        # Print reporting owner address if present
        addr_tag = ro.find('rptOwnerAddress')
        if addr_tag:
            addr_lines = []
            for part in ['rptOwnerStreet1','rptOwnerStreet2','rptOwnerCity','rptOwnerState','rptOwnerZipCode']:
                t = addr_tag.find(part)
                if t and t.text.strip():
                    addr_lines.append(t.text.strip())
            if addr_lines:
                print(f"    Address: {', '.join(addr_lines)}")

        print("------")

print('Done iterating form4 filings.')
